In [1]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime

In [2]:
pp_dep = pd.read_csv("pp_dep.csv", sep=";", dtype={"reg":str}, parse_dates=['period_date'])

In [3]:
pp_dep.period_date.unique()

array(['2021-01-31T00:00:00.000000000', '2021-02-28T00:00:00.000000000',
       '2020-11-30T00:00:00.000000000', '2020-12-31T00:00:00.000000000',
       '2021-03-31T00:00:00.000000000', '2021-04-30T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [4]:
pp_dep.indicateur.unique()

array(['Nombre de repas servis dans les restaurants universitaires au tarif à 1€ - RRU',
       'Nombre d’entrées en service civique - SCI', 'Nombre de PME - FAA',
       'Montant cumulé de l’investissement total ainsi déclenché - FAA',
       'Nombre d’entreprises - FAA', 'Nombre de PME - BPI',
       'Nombre d’entreprises - BPI',
       'Montant cumulé de l’investissement total ainsi déclenché - BPI',
       'Nombre de projets incluant une transformation de la ligne de production pour réduire son impact environnemental - BPI',
       'Nombre d’entreprises bénéficiaires du dispositif - MIR',
       'Nombre de PME bénéficiaires - MIR',
       'Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE',
       'Nombre d’entreprises bénéficiares - SBF',
       'Nombre de TPE/PME/ETI bénéficiaires - SBF',
       "Nombre d'accompagnements dispensés - FUM",
       'Nombre de TPE/PME/ETI bénéficiaires - APR',
       'Montant total des projets - SAC',
       'Nombre de TPE/PME/E

In [5]:
corres = {
    "Nombre de bâtiments de l’Etat retenus pour faire l’objet d’une rénovation énergétique" : 
        ['Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE'],
    "Nombre de bonus écologiques et de primes à la conversion versés " : 
        ['Nombre de primes à la conversion - DVP',
        'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables - BOE'],
    "Nombre de projets industriels soutenus" : [ 'Nombre d’entreprises - BPI',
                                                'Nombre d’entreprises bénéficiares - PIT', 
                                                'Nombre d’entreprises bénéficiaires - SAC'],
    "Nombre de chèques export et chèques VIE (volontariat international en entreprise" : 
        ['Nombre d’entreprises bénéficiares - SBF'],
    "Nombre de contrats d’apprentissage ayant bénéficié de l’aide exceptionnelle " : 
        ['Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP'],
    "Nombre d’aides versées pour l’embauche des jeunes" : 
        ["Nombre d'aides à l'embauche des jeunes - PEJ"],
    "Nombre de jeunes entrés en CIE (contrats initiative emploi) et PEC (parcours emploi compétences) " : 
        ['Entrées de jeunes en CIE - CIE', 'Entrées de jeunes en PEC - PEC']
}

In [6]:
pd.DataFrame.from_dict(corres, orient='index').to_csv("correspondances.csv")

In [7]:
list_indicateurs = list(itertools.chain.from_iterable(corres.values()))
len(list_indicateurs)

11

In [8]:
pp_dep_indics = pp_dep[pp_dep['indicateur'].isin(list_indicateurs)].reset_index(drop=True)
pp_dep_indics.drop('Unnamed: 0', axis=1, inplace=True)
pp_dep_indics.shape

(5459, 14)

In [22]:
def get_latest_date_for_indic_and_dep(indicateur, dep):
    df = pp_dep_indics[(pp_dep_indics['indicateur'] == indicateur) & 
                          (pp_dep_indics['dep'] == dep)]
    return df['period_date'].max()


pp_dep_to_export = pp_dep_indics.loc[pp_dep_indics.apply(lambda x: get_latest_date_for_indic_and_dep(x['indicateur'], x['dep']) == x['period_date'], axis=1)]
pp_dep_to_export.reset_index(drop=True, inplace=True)
pp_dep_to_export

,indicateur,state_id,period_date,period_month_tri,Date,valeur,departement,mesure,short_indic,short_mesure,dep,reg,libelle,region
0,Nombre d’entreprises - BPI,Valeur Actuelle,2021-03-31,Mar,Mars 2021,0.0,Île de Clipperton,Modernisation de la filière (BPI),Nombre d'entreprises,AAP industrie : Modernisation des filières aut...,989,989,Île de Clipperton,Île de Clipperton
1,Nombre d’entreprises - BPI,Valeur Actuelle,2021-03-31,Mar,Mars 2021,2.0,Charente,Modernisation de la filière (BPI),Nombre d'entreprises,AAP industrie : Modernisation des filières aut...,16,75,Charente,Nouvelle-Aquitaine
2,Nombre d’entreprises - BPI,Valeur Actuelle,2021-03-31,Mar,Mars 2021,9.0,Indre,Modernisation de la filière (BPI),Nombre d'entreprises,AAP industrie : Modernisation des filières aut...,36,24,Indre,Centre-Val de Loire
3,Nombre d’entreprises - BPI,Valeur Actuelle,2021-03-31,Mar,Mars 2021,24.0,Loire-Atlantique,Modernisation de la filière (BPI),Nombre d'entreprises,AAP industrie : Modernisation des filières aut...,44,52,Loire-Atlantique,Pays de la Loire
4,Nombre d’entreprises - BPI,Valeur Actuelle,2021-03-31,Mar,Mars 2021,3.0,Allier,Modernisation de la filière (BPI),Nombre d'entreprises,AAP industrie : Modernisation des filières aut...,03,84,Allier,Auvergne-Rhône-Alpes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,Nombre de contrats d’apprentissage bénéficiair...,Valeur Actuelle,2021-03-31,Mar,Mars 2021,2119.0,Dordogne,Apprentissage,Nombre de contrats d’apprentissage,Apprentissage,24,75,Dordogne,Nouvelle-Aquitaine
1197,Nombre de contrats d’apprentissage bénéficiair...,Valeur Actuelle,2021-03-31,Mar,Mars 2021,984.0,Gers,Apprentissage,Nombre de contrats d’apprentissage,Apprentissage,32,76,Gers,Occitanie
1198,Nombre de contrats d’apprentissage bénéficiair...,Valeur Actuelle,2021-03-31,Mar,Mars 2021,1670.0,Orne,Apprentissage,Nombre de contrats d’apprentissage,Apprentissage,61,28,Orne,Normandie
1199,Nombre de contrats d’apprentissage bénéficiair...,Valeur Actuelle,2021-03-31,Mar,Mars 2021,6161.0,Essonne,Apprentissage,Nombre de contrats d’apprentissage,Apprentissage,91,11,Essonne,Île-de-France


In [23]:
export = pd.pivot_table(pp_dep_to_export, columns=['dep'], index=['mesure', 'indicateur'])
export.columns = export.columns.get_level_values(1)
export['National'] = export.sum(axis=1)
export.reset_index(drop=False, inplace=True)
export.set_index('indicateur', inplace=True)

In [39]:
export = pd.pivot_table(pp_dep_to_export, columns=['dep', 'reg', 'departement', 'region'],
                        index=['mesure', 'indicateur'], values="valeur")
#export.columns = export.columns.get_level_values(1)
#export['National'] = export.sum(axis=1)
export.reset_index(inplace=True)
export["index"] = export["mesure"] + "/" + export["indicateur"]
export.set_index('index', inplace=True)
export.drop(columns=["mesure", "indicateur"], inplace=True)
export = export.T
export.fillna(0, inplace=True)
export = export.loc[~export.index.isin(["00"], level=1)]

/home/qdesrousseaux-adc/anaconda3/envs/propilot2pdf/lib/python3.9/site-packages/pandas/core/generic.py:4152: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [40]:
export["Nombre de bonus écologiques et de primes à la conversion versés"] = \
    export["Prime à la conversion des véhicules légers/Nombre de primes à la conversion - DVP"] + \
    export["Bonus électrique/Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables - BOE"]

export["Nombre de projets industriels soutenus"] = (
    export["Modernisation de la filière (BPI)/Nombre d’entreprises - BPI"] +
    export["Relocalisation : soutien aux projets industriels dans les territoires/Nombre d’entreprises bénéficiares - PIT"] +
    export["Relocalisation : sécurisation des approvisionnements critiques/Nombre d’entreprises bénéficiaires - SAC"])

export["Nombre de jeunes entrés en CIE et PEC"] = (
    export["PEC jeunes/Entrées de jeunes en PEC - PEC"] +
    export["CIE jeunes/Entrées de jeunes en CIE - CIE"]

)
export

,,,index,Apprentissage/Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP,Bonus électrique/Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables - BOE,CIE jeunes/Entrées de jeunes en CIE - CIE,Modernisation de la filière (BPI)/Nombre d’entreprises - BPI,PEC jeunes/Entrées de jeunes en PEC - PEC,Prime à l'embauche des jeunes/Nombre d'aides à l'embauche des jeunes - PEJ,Prime à la conversion des véhicules légers/Nombre de primes à la conversion - DVP,Relocalisation : soutien aux projets industriels dans les territoires/Nombre d’entreprises bénéficiares - PIT,Relocalisation : sécurisation des approvisionnements critiques/Nombre d’entreprises bénéficiaires - SAC,"Renforcement des subventions de Business France (chèque export, chèque VIE)/Nombre d’entreprises bénéficiares - SBF",Rénovation bâtiments Etats/Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE,Nombre de bonus écologiques et de primes à la conversion versés,Nombre de projets industriels soutenus,Nombre de jeunes entrés en CIE et PEC
dep,reg,departement,region,,,,,,,,,,,,,,
01,84,Ain,Auvergne-Rhône-Alpes,3303.0,1495.0,51.0,7.0,116.0,2181.0,1616.0,5.0,3.0,24.0,1.0,3111.0,15.0,167.0
02,32,Aisne,Hauts-de-France,2109.0,968.0,114.0,3.0,285.0,1230.0,1332.0,4.0,2.0,14.0,4.0,2300.0,9.0,399.0
03,84,Allier,Auvergne-Rhône-Alpes,1494.0,474.0,42.0,3.0,139.0,1124.0,1036.0,1.0,1.0,13.0,4.0,1510.0,5.0,181.0
04,93,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,875.0,379.0,13.0,0.0,71.0,424.0,627.0,6.0,0.0,9.0,3.0,1006.0,6.0,84.0
05,93,Hautes-Alpes,Provence-Alpes-Côte d'Azur,647.0,285.0,11.0,2.0,46.0,453.0,503.0,1.0,2.0,2.0,3.0,788.0,5.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,984,Terres australes et antarctiques françaises,Terres australes et antarctiques françaises,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
986,COM,Wallis-et-Futuna,Collectivités d’outre-mer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
987,COM,Polynésie française,Collectivités d’outre-mer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0


In [41]:
!mkdir exports

mkdir: impossible de créer le répertoire «exports»: Le fichier existe


### Rénovation bâtiments états

In [42]:
tcsv = export["Rénovation bâtiments Etats/Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE"]
tcsv.to_csv("exports/2_renovation_batiment_etat.csv", sep=";")

### Bonus écologique

In [43]:
tcsv = export["Nombre de bonus écologiques et de primes à la conversion versés"]
tcsv.to_csv("exports/3_bonus_ecologique.csv", sep=";")

### Soutien industriel 

In [44]:
tcsv = export["Nombre de projets industriels soutenus"]
tcsv.to_csv("exports/4_soutien_industriel.csv", sep=";")

### Soutien Business France

In [45]:
tcsv = export["Renforcement des subventions de Business France (chèque export, chèque VIE)/Nombre d’entreprises bénéficiares - SBF"]
tcsv.to_csv("exports/5_cheque_export_vie_sbf.csv", sep=";")

### Apprentissage

In [46]:
tcsv = export["Apprentissage/Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP"]
tcsv.to_csv("exports/6_apprentissage.csv", sep=";")

### Emploi Jeune

In [47]:
tcsv = export["Prime à l'embauche des jeunes/Nombre d'aides à l'embauche des jeunes - PEJ"]
tcsv.to_csv("exports/7_embauche_jeunes.csv", sep=";")

### CIE/PEC

In [48]:
tcsv = export["Nombre de jeunes entrés en CIE et PEC"]
tcsv.to_csv("exports/8_cie_pec.csv", sep=";")